In [10]:
import yaml
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text

In [2]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)
    configSource = config['source']
    configDestination = config['destination']

urlSource = f"{configSource['driver']}://{configSource['user']}:{configSource['password']}@{configSource['host']}:{configSource['port']}/{configSource['db']}"
urlDestination = f"{configDestination['driver']}://{configDestination['user']}:{configDestination['password']}@{configDestination['host']}:{configDestination['port']}/{configDestination['db']}"

engineSource = create_engine(urlSource)
engineDestination = create_engine(urlDestination)

In [11]:
type_novelty = pd.read_sql_table('mensajeria_tiponovedad', engineSource)
type_novelty.rename(columns={
  "id": "type_novelty_id", 
  "nombre": "name"
  }, inplace=True)
type_novelty["saved"] = pd.to_datetime('now')

type_novelty

,type_novelty_id,name,saved
0,2,No puedo continuar,2024-10-22 20:47:21.133404
1,1,Novedades del servicio,2024-10-22 20:47:21.133404


In [12]:
type_novelty.to_sql("dim_type_novelty", engineDestination, index=False, if_exists="replace")

alter_table_query = """
    ALTER TABLE dim_type_novelty
    ADD CONSTRAINT pk_dim_type_novelty PRIMARY KEY (type_novelty_id);
"""

with engineDestination.connect() as connection:
    connection.execute(text(alter_table_query))